In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
# Read the parquet file into a DataFrame

df = pd.read_parquet('/Users/michaelb/Project 01/stock_data.parquet')
print(df.head())

  symbol                          name    price  changesPercentage  change  \
0   LULU      Lululemon Athletica Inc.  457.810             0.8792   3.990   
1   CROX                   Crocs, Inc.  100.170            -1.2909  -1.310   
2   SHAK              Shake Shack Inc.   77.135             2.0844   1.575   
3   COST  Costco Wholesale Corporation  697.525             0.5282   3.665   
4    WBD  Warner Bros. Discovery, Inc.   10.185             1.6467   0.165   

    dayLow   dayHigh  yearHigh  yearLow     marketCap  priceAvg50  \
0  454.020  459.4900    516.39   286.58   60113561072    477.4530   
1   99.750  103.3199    151.32    74.00    6066956322    100.3214   
2   76.110   77.5700     80.58    52.01    3262993464     68.4142   
3  691.500  697.5250    705.52   465.33  309511373200    647.1492   
4   10.055   10.3950     16.34     9.27   24836835450     11.0308   

   priceAvg200   volume    eps                 timestamp  
0    404.98395   264212   7.85 2024-02-01 15:10:07+00:00 

In [77]:
# Describe the DataFrame

description = df.describe()

description

,price,changesPercentage,change,dayLow,dayHigh,yearHigh,yearLow,marketCap,priceAvg50,priceAvg200,volume,eps,price_change,changesPercentage_diff,price_diff,daily_volatility,price_to_avg50_ratio,opening_prices
count,26190.000000,26190.000000,26190.000000,26190.000000,26190.000000,26190.000000,26190.000000,2.619000e+04,26190.000000,26190.000000,2.619000e+04,26190.000000,26181.000000,26181.000000,26181.000000,26190.00000,26190.000000,0.0
mean,280.079759,1.282090,3.135314,276.340845,281.199815,304.506889,177.004444,2.444774e+11,264.896471,239.347622,5.876501e+06,8.091430,0.000004,0.000418,0.000990,4.85897,1.033985,NaN
std,258.201778,1.466611,4.295011,254.659591,258.663660,265.072428,159.774949,4.696782e+11,237.272729,206.990702,1.023326e+07,8.517710,0.000275,0.027962,0.070442,4.39351,0.087766,NaN
min,10.075000,-2.542400,-2.710000,10.055000,10.395000,16.340000,9.270000,8.429513e+07,11.030800,11.816700,5.510000e+02,-1.940000,-0.004286,-0.449100,-0.950000,0.01000,0.913352,NaN
25%,77.345000,0.067500,0.020000,76.110000,77.600000,80.580000,52.010000,6.063777e+09,68.414200,66.521250,4.298040e+05,1.102868,0.000000,0.000000,0.000000,1.49000,0.960220,NaN
50%,100.660000,1.297000,1.720000,98.890000,103.319900,151.320000,87.230000,6.033219e+10,100.321400,103.133250,9.115440e+05,7.560000,0.000000,0.000000,0.000000,4.42990,1.003375,NaN
75%,461.475000,2.124250,5.550000,454.020000,462.350000,516.390000,286.580000,1.440822e+11,477.453000,436.910550,2.843463e+06,10.850000,0.000000,0.000000,0.000000,7.19000,1.087393,NaN
max,705.300000,4.790400,16.545000,691.500000,705.300000,705.520000,465.330000,1.560583e+12,647.149200,566.326350,4.134359e+07,27.560000,0.004304,0.449100,0.928700,15.40280,1.224462,NaN


In [85]:
df['timestamp'] = pd.to_datetime(df['timestamp']) # Ensure 'timestamp' is a datetime column
minute_groups = df.set_index('timestamp').groupby([pd.Grouper(freq='T'), 'symbol'])
median_prices_per_minute = minute_groups['price'].median()
df['timestamp'] = pd.to_datetime(df['timestamp']) 
minute_groups = df.set_index('timestamp').groupby([pd.Grouper(freq='T'), 'symbol'])      
median_prices_per_minute = minute_groups['price'].median().unstack()
topmedianpricechange = median_prices_per_minute.idxmax()

print(topmedianpricechange)

symbol
COST   2024-02-01 17:55:00+00:00
CROX   2024-02-01 18:00:00+00:00
DJIA   2024-02-01 17:53:00+00:00
LMT    2024-02-01 17:06:00+00:00
LULU   2024-02-01 20:06:00+00:00
NVDA   2024-02-01 20:34:00+00:00
PM     2024-02-01 21:00:00+00:00
SHAK   2024-02-01 20:49:00+00:00
WBD    2024-02-01 19:58:00+00:00
dtype: datetime64[ns, UTC]


In [79]:
## Top price movements (minute to minute) for the trading day

df['changesPercentage_diff'] = df.groupby('symbol')['changesPercentage'].diff()
df['price_diff'] = df.groupby('symbol')['price'].diff()
top_movements = df.sort_values(by='price_diff', key=abs, ascending=False).head(10)
print(top_movements[['symbol', 'name', 'price', 'changesPercentage_diff', 'price_diff', 'timestamp']].to_string())

      symbol                      name     price  changesPercentage_diff  price_diff                 timestamp
801     LULU  Lululemon Athletica Inc.  456.0700                 -0.2093     -0.9500 2024-02-01 15:22:10+00:00
4550    NVDA        NVIDIA Corporation  619.7687                  0.1510      0.9287 2024-02-01 16:18:09+00:00
23153   NVDA        NVIDIA Corporation  630.1900                  0.1407      0.8657 2024-02-01 20:55:03+00:00
13910   NVDA        NVIDIA Corporation  626.1500                 -0.1404     -0.8640 2024-02-01 18:37:36+00:00
4496    NVDA        NVIDIA Corporation  618.1600                 -0.1252     -0.7704 2024-02-01 16:17:20+00:00
99      LULU  Lululemon Athletica Inc.  457.9700                  0.1631      0.7400 2024-02-01 15:11:47+00:00
4523    NVDA        NVIDIA Corporation  618.8550                  0.1156      0.7110 2024-02-01 16:17:45+00:00
3029    NVDA        NVIDIA Corporation  620.0502                 -0.1156     -0.7109 2024-02-01 15:55:26+00:00
2

In [75]:
## Volatility Analysis

pricehigh = df.groupby('symbol')['price'].max()
pricelow = df.groupby('symbol')['price'].min()
pricediff = pricehigh - pricelow
opening_prices = df.groupby('symbol')['price'].first()
volatility = pricediff / opening_prices *100

print(volatility.sort_values(ascending=False))

## the difference between the highest price and the lowest price, relative to the opening price for the day

symbol
WBD     4.172803
SHAK    3.422571
CROX    2.810223
NVDA    2.290303
PM      1.734417
LULU    1.554138
COST    1.116806
LMT     1.007285
DJIA    0.495272
Name: price, dtype: float64


In [78]:
# Identify stocks that are trading highest above their 50 day moving average

print(df[['symbol', 'price_to_avg50_ratio']].drop_duplicates(subset='symbol', keep='first').sort_values(by='price_to_avg50_ratio', ascending=False).head(10))

  symbol  price_to_avg50_ratio
5   NVDA              1.205383
2   SHAK              1.127471
3   COST              1.077843
8   DJIA              1.016660
1   CROX              0.998491
7     PM              0.987694
0   LULU              0.958859
6    LMT              0.951901
4    WBD              0.923324
